In [24]:
import pandas as pd
import numpy as np
import sklearn


Processing c:\users\carso\appdata\local\pip\cache\wheels\f5\5e\9f\dbce0d6a89f44b3f30fba0a9b1b24a288882ea2e235e515d7b\html_parser-0.2-cp36-none-any.whl


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Carso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Carso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Carso\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Carso\AppData\Roaming\nltk_data...


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>


[nltk_data]   Package wordnet is already up-to-date!


Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), ('<', 'n'), ('/p', 'n'), ('>', 'n')]
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  < p > the circus dog in a plissé skirt jump over python who be n't that large , just 3 foot long. < /p >
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:    p   The circus dog in a plissé skirt jumped over Python who was n t that large   just 3 feet long     p  
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  < p > The circus dog pliss

In [38]:
reviews_raw = pd.read_csv("C:/Users/Carso/Desktop/DAT562/Code/Code/Video_Games.csv")
reviews_notnull = reviews_raw.dropna(subset = ['reviewText']).reset_index(drop = True)
%run ./Text_Normalization_Function.ipynb

In [286]:
reviews_norm = pd.read_csv('normalized_reviews.csv')
stopword_list = stopword_list + ['mr', 'mrs', 'come', 'go', 'get',
                                 'tell', 'listen', 'one', 'two', 'three',
                                 'four', 'five', 'six', 'seven', 'eight',
                                 'nine', 'zero', 'join', 'find', 'make',
                                 'say', 'ask', 'tell', 'see', 'try', 'back',
                                 'also','would','game','play']
new_reviews_norm = []
for index, text in enumerate(reviews_norm['0'].to_list()):
    try:
        text = remove_stopwords(text)
    except:
        text = text
    new_reviews_norm.append(text)

In [287]:
reviews_notnull['normalized'] = new_reviews_norm

### For a particular game, perform topic modeling for its negative and positive reviews, respectively, and see what people are talking about in both positive and negative reviews
#### First try to find out games with most reviews

In [157]:
top_games = pd.DataFrame(reviews_notnull.groupby(by= 'asin').reviewText.count()).sort_values(by = "reviewText", ascending = False)
top_games.reset_index(drop = False, inplace = True)
top_games.head(5)

,asin,reviewText
0,B00BGA9WK2,801
1,B0015AARJI,652
2,B007CM0K86,517
3,B002VBWIP6,465
4,B0009VXBAQ,447


In [238]:
#Most Top reviewed games have too few negative reviews to do topic modeling
game_top_asin = top_games.asin[3]
game_top = reviews_notnull.loc[(reviews_notnull["asin"] == game_top_asin),:]
reviews_notnull.loc[(reviews_notnull["asin"] == game_top_asin),:].groupby(by = 'overall').asin.count()

overall
1      9
2      4
3     14
4     39
5    399
Name: asin, dtype: int64

### Now find out the most controversial game among top 100 reviewed games (number of positive and negative reviews are about the same)

In [198]:
#First check how many games have over 150 reviews
top_games.loc[(top_games.reviewText >=150),"asin"].count()

144

In [206]:
top_100_games_asin = top_games.iloc[:100,0].to_list()
p_nratio_list = []
for asin in top_100_games_asin:
    negative_reviews = reviews_notnull.loc[(reviews_notnull["asin"] == asin)&(reviews_notnull['overall']==1),:].asin.count()\
    +reviews_notnull.loc[(reviews_notnull["asin"] == asin)&(reviews_notnull['overall']==2),:].asin.count()
    positive_reviews = reviews_notnull.loc[(reviews_notnull["asin"] == asin)&(reviews_notnull['overall']==4),:].asin.count()\
    +reviews_notnull.loc[(reviews_notnull["asin"] == asin)&(reviews_notnull['overall']==5),:].asin.count()
    p_nratio = positive_reviews/negative_reviews
    p_nratio_list.append(p_nratio)
top_100_games_pn = pd.DataFrame(data = {"game":top_100_games_asin, "p_nratio":p_nratio_list})

In [234]:
top_100_games_pn.sort_values(by = 'p_nratio', inplace = True)
top_100_games_pn.reset_index(drop = True, inplace = True)
top_100_games_pn.head()

,game,p_nratio
0,B000FKBCX4,0.160920
1,B007FTE2VW,0.237762
2,B00178630A,0.653465
3,B002I098JE,1.131579
4,B002I0JGDM,1.389831


In [289]:
game_contro_asin = top_100_games_pn.game[3]
game_contro = reviews_notnull.loc[(reviews_notnull["asin"] == game_contro_asin),:]
reviews_notnull.loc[(reviews_notnull["asin"] == game_1_asin),:].groupby(by = 'overall').asin.count()
#activision call of duty

overall
1    53
2    23
3    32
4    33
5    53
Name: asin, dtype: int64

#### Activision: Call of Duty is the most controversial game with a total of 194 reviews. The ratio of positive reviews and negative reviews is 1.13
#### Now perform topic modeling on its positive and negative reviews, respectively.

In [290]:
#Prepare corpuses of positive and negatives reviews
game_contro_positive = game_contro.loc[(game_contro.overall==5),'normalized'].tolist()
game_contro_positive+=game_contro.loc[(game_contro.overall==4),'normalized'].tolist()
game_contro_negative = game_contro.loc[(game_contro.overall==2),'normalized'].tolist()
game_contro_negative+=game_contro.loc[(game_contro.overall==1),'normalized'].tolist()

In [262]:
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation 

In [263]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

## Positive reviews topic modeling

In [350]:
bow_vectorizer_pos = CountVectorizer()
bow_contro_positive = bow_vectorizer_pos.fit_transform(game_contro_positive)
pd.DataFrame(data = bow_contro_positive.todense(), columns = bow_vectorizer.get_feature_names())

,1st,60fps,6v6,ability,able,absolute,absolutely,absolutley,accessible,accuracy,...,xp,ya,year,yelping,yep,yes,yet,youll,zombie,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,2,1,0,0,0
82,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
83,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [321]:
#find out best models for positive reviews by maximizing log-likelyhood score
#first by changing number of topics
def get_logscore_num_topics(bow,lower_limit, higher_limit, doc_topic_prior, topic_word_prior):
    score_dic = {}
    for i in range(lower_limit, higher_limit+1):
        model = LatentDirichletAllocation(n_components=i, max_iter=500,
                                                doc_topic_prior = doc_topic_prior,
                                                topic_word_prior = topic_word_prior).fit(bow)
        score_dic[i] =  model.score(bow)
    return pd.DataFrame.from_dict(score_dic, orient = 'index')

In [322]:
logscore_num_topics= get_logscore_num_topics(bow_contro_positive,1, 8, 0.25,0.25)
logscore_num_topics

,0
1,-27153.365714
2,-27249.471687
3,-27161.359031
4,-27231.319503
5,-27420.033862
6,-27311.352755
7,-27386.622330
8,-27664.010226


In [323]:
#then adjust hyperparameters
def get_logscore_params(bow, best_number_of_topics,lower_limit, higher_limit,step):
    score_dic = {}
    for i in list(np.linspace(lower_limit, higher_limit, step)):
        for j in list(np.linspace(lower_limit, higher_limit, step)):
            model = LatentDirichletAllocation(n_components=best_number_of_topics, max_iter=500,
                                                doc_topic_prior = i,
                                                topic_word_prior = j).fit(bow)
            score_dic[(i,j)] =  model.score(bow)
    return pd.DataFrame.from_dict(score_dic, orient = 'index')

In [324]:
logscore_params = get_logscore_params(bow_contro_positive,3, 0.05,0.5,10)
logscore_params.sort_values(by = 0,ascending = False, inplace = True)
logscore_params

,0
"(0.05, 0.05)",-28692.700300
"(0.05, 0.1)",-27858.757258
"(0.05, 0.15000000000000002)",-27490.528651
"(0.05, 0.2)",-27315.424920
"(0.05, 0.25)",-27127.966356
...,...
"(0.5, 0.3)",-27114.911369
"(0.5, 0.35000000000000003)",-27079.669262
"(0.5, 0.4)",-26955.345881
"(0.5, 0.45)",-27001.386589


In [328]:
logscore_params

,0
"(0.15000000000000002, 0.5)",-26855.398748
"(0.05, 0.45)",-26863.165376
"(0.2, 0.45)",-26892.057329
"(0.4, 0.5)",-26911.536760
"(0.05, 0.5)",-26922.404596
...,...
"(0.1, 0.05)",-28719.789877
"(0.3, 0.05)",-28723.809652
"(0.25, 0.05)",-28738.305065
"(0.35000000000000003, 0.05)",-28752.688627


In [349]:
#Best model for positive reviews
lda_contro_positive_3 = LatentDirichletAllocation(n_components=3, max_iter=500,
                                                doc_topic_prior =0.15 ,
                                                topic_word_prior = 0.5).fit(bow_contro_positive)
display_topics(lda_contro_positive_3, bow_vectorizer_pos.get_feature_names(), no_top_words)

Topic 0:
good like duty call cod multiplayer ghost campaign great series fun new buy map story
Topic 1:
multiplayer cod squad like mode good time different new campaign fun much people want still
Topic 2:
like cod map ops black ghost people really good great thing player lot look well


In [330]:
#Visualize topics of positive reviews
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_contro_positive_3, bow_contro_positive, bow_vectorizer_pos, mds='tsne')

PreparedData(topic_coordinates=                 x            y  topics  cluster       Freq
topic                                                      
2     -1015.700562  1338.981201       1        1  38.278352
1      -387.996002  -373.316162       2        1  31.624505
0     -2184.745605   -60.565800       3        1  30.097143, topic_info=     Category       Freq     Term      Total  loglift  logprob
1063  Default  18.000000    squad  18.000000  30.0000   30.000
500   Default  43.000000     good  43.000000  29.0000   29.000
789   Default  20.000000      ops  20.000000  28.0000   28.000
417   Default  10.000000      fan  10.000000  27.0000   27.000
35    Default   6.000000       ai   6.000000  26.0000   26.000
...       ...        ...      ...        ...      ...      ...
1224   Topic3   4.231305     want  11.837828   0.1720   -5.637
673    Topic3   4.231080     look  11.921867   0.1648   -5.637
53     Topic3   4.231164  another  12.328745   0.1313   -5.637
1135   Topic3   4.231240    thing  15.343252  -0.0874   -5.637
115    Topic3   4.231169     best  16.409177  -0.1546   -5.637

[237 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
10        3  0.866789  achievement
13        1  0.201234       action
13        2  0.201234       action
13        3  0.603701       action
26        2  0.860717    advantage
...     ...       ...          ...
1276      3  0.436152         year
1279      1  0.643339          yes
1280      1  0.206546          yet
1280      2  0.619638          yet
1280      3  0.206546          yet

[319 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

#### Models with 2 and 4 topics for positive reviews

In [352]:
lda_contro_positive_4 = LatentDirichletAllocation(n_components=4, max_iter=500,
                                                doc_topic_prior =0.15 ,
                                                topic_word_prior = 0.5).fit(bow_contro_positive)
display_topics(lda_contro_positive_4, bow_vectorizer_pos.get_feature_names(), no_top_words)

Topic 0:
good cod like multiplayer map really campaign black ops next enjoy great best people year
Topic 1:
fun buy like new actually xbox recommend call enjoy good gun drop multiplayer campaign time
Topic 2:
like map duty call ops black ghost people cod player great lot good love mode
Topic 3:
cod like multiplayer good mode squad ghost new story great another time campaign really duty


In [362]:
pyLDAvis.sklearn.prepare(lda_contro_positive_4, bow_contro_positive, bow_vectorizer_pos, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      269.562408 -124.803398       1        1  33.983705
0        1.782994 -295.079681       2        1  33.221217
2      220.811432 -343.832123       3        1  25.913300
1       50.533993  -76.050934       4        1   6.881778, topic_info=     Category       Freq      Term      Total  loglift  logprob
473   Default  16.000000       fun  16.000000  30.0000  30.0000
1087  Default  11.000000     story  11.000000  29.0000  29.0000
718   Default  20.000000      mode  20.000000  28.0000  28.0000
16    Default   6.000000  actually   6.000000  27.0000  27.0000
164   Default  15.000000       buy  15.000000  26.0000  26.0000
...       ...        ...       ...        ...      ...      ...
213    Topic4   1.049773       cod  43.006111  -1.0365  -5.5553
122    Topic4   0.750058       bit   7.608488   0.3594  -5.8915
348    Topic4   0.750012      duty  25.015188  -0.8309  -5.8916
783    Topic4   0.750004    online  13.777318  -0.2344  -5.8916
1200   Topic4   0.749995       use  15.001483  -0.3195  -5.8916

[296 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
10        2  0.809557  achievement
13        1  0.202721       action
13        2  0.202721       action
13        3  0.608162       action
16        1  0.160151     actually
...     ...       ...          ...
1276      2  0.433542         year
1276      3  0.072257         year
1280      1  0.716963          yet
1280      2  0.102423          yet
1280      3  0.204847          yet

[391 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])

In [353]:
lda_contro_positive_2 = LatentDirichletAllocation(n_components=2, max_iter=500,
                                                doc_topic_prior =0.15 ,
                                                topic_word_prior = 0.5).fit(bow_contro_positive)
display_topics(lda_contro_positive_2, bow_vectorizer_pos.get_feature_names(), no_top_words)

Topic 0:
good like cod ghost map call duty black multiplayer ops new fun great campaign enjoy
Topic 1:
cod squad like multiplayer mode people thing time really great want year lot campaign player


In [361]:
pyLDAvis.sklearn.prepare(lda_contro_positive_2, bow_contro_positive, bow_vectorizer_pos, mds='tsne')

PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
0     -60.312317  141.369629       1        1  64.513417
1      60.312561 -141.369141       2        1  35.486583, topic_info=     Category       Freq    Term      Total  loglift  logprob
1063  Default  19.000000   squad  19.000000  30.0000  30.0000
35    Default   6.000000      ai   6.000000  29.0000  29.0000
139   Default   7.000000     bot   7.000000  28.0000  28.0000
1224  Default  12.000000    want  12.000000  27.0000  27.0000
165   Default  30.000000    call  30.000000  26.0000  26.0000
...       ...        ...     ...        ...      ...      ...
500    Topic2   6.435175    good  49.971052  -1.0136  -5.3824
164    Topic2   5.161264     buy  19.899274  -0.3135  -5.6030
869    Topic2   4.473271  pretty  10.447957   0.1877  -5.7461
685    Topic2   5.161338     map  33.442258  -0.8326  -5.6030
755    Topic2   4.843414     new  25.525902  -0.6261  -5.6666

[163 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
10        2  0.943949  achievement
13        1  0.200806       action
13        2  0.803225       action
35        2  0.960338           ai
44        1  1.018072     although
...     ...       ...          ...
1271      2  0.106416         xbox
1276      1  0.540762         year
1276      2  0.473166         year
1282      1  0.882898       zombie
1282      2  0.098100       zombie

[177 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

# 2 topics for positive reviews: 
1. The general gameplay, including weapon system, graphic, map, online system, story
2. The Bots and AIs in online multiplayer mode

## Negative reviews topic modeling

In [354]:
#find out best models for negative reviews by maximizing log-likelyhood score
bow_vectorizer_neg = CountVectorizer()
bow_contro_negative = bow_vectorizer_neg.fit_transform(game_contro_negative)
logscore_num_topics_neg= get_logscore_num_topics(bow_contro_negative,1, 8, 0.25,0.25)
logscore_num_topics_neg

,0
1,-51458.681346
2,-51412.096098
3,-51501.359669
4,-51633.078048
5,-51630.134429
6,-51696.370151
7,-51749.939186
8,-51942.208139


In [332]:
logscore_params_neg = get_logscore_params(bow_contro_negative,2, 0.05,0.5,10)
logscore_params_neg.sort_values(by = 0,ascending = False, inplace = True)
logscore_params_neg

,0
"(0.5, 0.5)",-50949.058148
"(0.1, 0.5)",-50954.733323
"(0.05, 0.5)",-50974.332324
"(0.15000000000000002, 0.5)",-51020.690347
"(0.25, 0.5)",-51025.706970
...,...
"(0.35000000000000003, 0.05)",-53727.071807
"(0.1, 0.05)",-53743.933599
"(0.2, 0.05)",-53746.986975
"(0.3, 0.05)",-53749.081668


In [355]:
#Best model for negative reviews
lda_contro_negative_2 = LatentDirichletAllocation(n_components=2, max_iter=500,
                                                doc_topic_prior =0.1 ,
                                                topic_word_prior = 0.5).fit(bow_contro_negative)
display_topics(lda_contro_negative_2, bow_vectorizer_neg.get_feature_names(), no_top_words)

Topic 0:
like map cod call duty ghost good bad look time even mode new every multiplayer
Topic 1:
terrible bad map spawn call good cod duty black kill buy ops like really player


In [358]:
#Visualize topics of negative reviews
pyLDAvis.sklearn.prepare(lda_contro_negative_2, bow_contro_negative, bow_vectorizer_neg, mds='tsne')

PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
0     -60.312317  141.369629       1        1  63.153177
1      60.312561 -141.369141       2        1  36.846823, topic_info=     Category       Freq      Term      Total  loglift  logprob
1727  Default  38.000000  terrible  38.000000  30.0000  30.0000
1612  Default  34.000000     spawn  34.000000  29.0000  29.0000
1516  Default  18.000000    series  18.000000  28.0000  28.0000
146   Default  56.000000       bad  56.000000  27.0000  27.0000
188   Default  32.000000     black  32.000000  26.0000  26.0000
...       ...        ...       ...        ...      ...      ...
1936   Topic2   9.146907      year  22.736680   0.0878  -5.6699
1754   Topic2  11.342161      time  38.933524  -0.2349  -5.4548
572    Topic2  10.610410     every  34.907281  -0.1925  -5.5215
1153   Topic2  10.610405       new  35.730904  -0.2158  -5.5215
1738   Topic2   9.146904     thing  26.031171  -0.0475  -5.6699

[163 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
113       2  1.009100  approximately
146       1  0.511650            bad
146       2  0.476364            bad
184       1  0.653063            big
184       2  0.345739            big
...     ...       ...            ...
1913      2  0.760446          world
1931      1  0.373696           xbox
1931      2  0.622827           xbox
1936      1  0.615745           year
1936      2  0.395836           year

[178 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

#### Models with 3 and 4 topics for negative reviews

In [356]:
lda_contro_negative_3 = LatentDirichletAllocation(n_components=3, max_iter=500,
                                                doc_topic_prior =0.1 ,
                                                topic_word_prior = 0.5).fit(bow_contro_negative)
display_topics(lda_contro_negative_3, bow_vectorizer_neg.get_feature_names(), no_top_words)

Topic 0:
map terrible call duty like ghost new bad cod kill player system every weapon time
Topic 1:
cod spawn bad map buy money take really terrible could star sniper player modded think
Topic 2:
like map good multiplayer call bad even black look duty mode cod ops time kill


In [359]:
pyLDAvis.sklearn.prepare(lda_contro_negative_3, bow_contro_negative, bow_vectorizer_neg, mds='tsne')

PreparedData(topic_coordinates=                 x            y  topics  cluster       Freq
topic                                                      
2     -1015.700562  1338.981201       1        1  44.295533
0      -387.996002  -373.316162       2        1  40.072185
1     -2184.745605   -60.565800       3        1  15.632282, topic_info=     Category       Freq      Term      Total  loglift  logprob
311   Default  54.000000       cod  54.000000  30.0000  30.0000
1727  Default  36.000000  terrible  36.000000  29.0000  29.0000
1612  Default  30.000000     spawn  30.000000  28.0000  28.0000
1110  Default  20.000000     money  20.000000  27.0000  27.0000
1103  Default   5.000000    modded   5.000000  26.0000  26.0000
...       ...        ...       ...        ...      ...      ...
745    Topic3   4.026254      good  46.597647  -0.5929  -5.6330
188    Topic3   3.651285     black  30.837525  -0.2778  -5.7308
996    Topic3   4.026245      like  68.517824  -0.9784  -5.6330
511    Topic3   3.655815      duty  51.754303  -0.7944  -5.7295
250    Topic3   3.635338      call  57.087861  -0.8981  -5.7352

[231 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
14        2  0.917865       able
34        3  0.476442     across
78        1  0.120339    already
78        2  0.722032    already
78        3  0.120339    already
...     ...       ...        ...
1936      2  0.323785       year
1936      3  0.092510       year
1940      3  0.476442  yesterday
1944      1  0.882569     zombie
1944      2  0.098063     zombie

[340 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

In [357]:
lda_contro_negative_4 = LatentDirichletAllocation(n_components=4, max_iter=500,
                                                doc_topic_prior =0.1 ,
                                                topic_word_prior = 0.5).fit(bow_contro_negative)
display_topics(lda_contro_negative_4, bow_vectorizer_neg.get_feature_names(), no_top_words)

Topic 0:
terrible map bad good like cod multiplayer call take even duty fun time ghost big
Topic 1:
mw2 mode map like cod pretty mw3 perk look black far multiplayer enemy campaign think
Topic 2:
call duty map like player kill ghost bad even new time system every well weapon
Topic 3:
cod like buy spawn map good bad player black time kill ops really ghost mode


In [360]:
pyLDAvis.sklearn.prepare(lda_contro_negative_4, bow_contro_negative, bow_vectorizer_neg, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      269.562408 -124.803398       1        1  37.132972
0        1.782994 -295.079681       2        1  28.598398
3      220.811432 -343.832123       3        1  27.487710
1       50.533993  -76.050934       4        1   6.780921, topic_info=     Category       Freq      Term      Total  loglift  logprob
1727  Default  35.000000  terrible  35.000000  30.0000  30.0000
1133  Default   7.000000       mw2   7.000000  29.0000  29.0000
1104  Default  24.000000      mode  24.000000  28.0000  28.0000
1261  Default  15.000000      perk  15.000000  27.0000  27.0000
1135  Default  10.000000       mw3  10.000000  26.0000  26.0000
...       ...        ...       ...        ...      ...      ...
1880   Topic4   1.173458    weapon  19.327488  -0.1105  -6.0307
1739   Topic4   1.173460     think  19.653355  -0.1272  -6.0307
1121   Topic4   1.173459      much  24.084616  -0.3306  -6.0307
1711   Topic4   1.173459      take  25.712695  -0.3960  -6.0307
568    Topic4   1.173459      even  35.846081  -0.7282  -6.0307

[322 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
14        1  0.880457        able
63        2  0.779678          ai
85        4  0.647821   amazingly
96        2  0.779678      animal
100       4  0.647821  antagonist
...     ...       ...         ...
1936      3  0.348179        year
1936      4  0.049740        year
1944      1  0.227996      zombie
1944      2  0.683989      zombie
1944      4  0.113998      zombie

[515 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])

# 3 topics for negative reviews
1. The spawning system, the sound, the story
2. The weapon system
3. The control